In [30]:
#imports
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
import os
from os import listdir, path

In [31]:
paths = [
    "Resultados",
    "Data"
]
def crearPaths(paths=paths):
    for p in paths:    
        if not path.exists(f"{os.getcwd()}/{p}"):
            os.mkdir(f"{os.getcwd()}/{p}")
crearPaths()

In [32]:
#correr experimentos (ver como hay que modificar con lo de usar python y c++ juntos)
def correr_experimento(archivo_training, archivo_testing, archivo_salida, k, metodo, nitter, epsilon, alfa):
        
    # Crear proceso para ejecutar el codigo.
    if metodo == 0:
        process = subprocess.Popen(["./build/tp2", archivo_training, archivo_testing, archivo_salida, k, metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    else:
        process = subprocess.Popen(["./build/tp2", archivo_training, archivo_testing, archivo_salida, k, metodo, nitter, epsilon, alfa], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar y leer salida de STDERR con el tiempo de ejecución.
    stdout, stderr = process.communicate() # communicate() devuelve una tupla (stdout, stderr)

    tiempo_de_ejecucion = str.splitlines(stderr)[0]
    #print(clog)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion con el metodo {'kNN' if metodo == 0 else 'PCA'} con los parámetros {archivo_training}, {archivo_testing}, k:{k} {', nitter:'+nitter if metodo == 1 else ''} {', epsilon:'+epsilon if metodo == 1 else ''} {', alfa:'+alfa if metodo == 1 else ''}.")
    return float(tiempo_de_ejecucion)


In [33]:
class Experimento:
    def __init__(self, trainingDataset, testingDataset, outputFile, metodo, vecinos, num_iter, alfa, epsilon):
        self.trainingDataset = trainingDataset
        self.testingDataset = testingDataset
        self.outputFile = outputFile
        self.metodo = metodo
        self.vecinos = vecinos
        self.num_iter = num_iter
        self.alfa = alfa
        self.epsilon = epsilon

    def display(self):
        display(f"Archivo De Entrenamiento: {self.trainingDataset}")
        display(f"Archivo De Prueba: {self.testingDataset}")
        display(f"Archivo De Salida: {self.outputFile}")
        display(f"Metodo: {self.metodo}")
        display(f"Vecinos: {self.vecinos}")
        display(f"Numero De Iteraciones: {self.num_iter}")
        display(f"Alfa: {self.alfa}")
        display(f"Epsilon: {self.epsilon}")

In [41]:
def reduceDataset(n):
    dataset = pd.read_csv("train.csv")
    reducedDataset = resample(dataset, replace = False, n_samples = n)
    return reducedDataset


def splitDataset(dataset, testSize=0.3): #default 70-30 como el fernet
    train, test = train_test_split(dataset, test_size=testSize);
    train.to_csv(path_or_buf="Data/random_train.csv", index=False)
    test.to_csv(path_or_buf="Data/random_validate.csv", index=False)
    return[train, test]


#obsoleta
def k_foldDataset(dataset, k):
    kf = KFold(n_splits=k, shuffle=True)
    fold_indices = kf.split(X=dataset)
    for key, indices in enumerate(fold_indices, 1):
        train_indices = indices[0]
        #test_indices = indices[1]
        fold_train_dataset = dataset.filter(items=train_indices, axis=0) 
        #fold_test_dataset = dataset.filter(items=test_indices, axis=0) 
        fold_train_dataset.to_csv(path_or_buf=f"Data/{k}_fold-{key}.csv", index=False)
        #fold_test_dataset.to_csv(path_or_buf=f"Data/{k}_fold-{key}-test.csv", index=False)
    

In [50]:
#si se corre dos veces sobre la misma cantidad de folds pisa los archivos
def prepareDataSet(size, folds):
    reducedDataset = reduceDataset(size);
    kf = KFold(n_splits=folds, shuffle=True)
    fold_splits = kf.split(X=reducedDataset)

    for key, split in enumerate(fold_splits, 1):
        training_indices = split[0]
        validate_indices = split[1]

        trainDataset = reducedDataset.iloc[training_indices]
        validateDataset = reducedDataset.iloc[validate_indices]
        
        trainDataset.to_csv(path_or_buf=f"Data/{folds}_fold-SET-{key}-train.csv", index=False)
        validateDataset.to_csv(path_or_buf=f"Data/{folds}_fold-SET-{key}-validate.csv", index=False)
                

<class 'pandas.core.frame.DataFrame'>


In [58]:
experimentos=[]

In [59]:
#Generar experimentos para busqueda de mejor k de knn sin pca en un 5-fold del dataset reducido
def generarExperimentosKnn(folds, maxK, step):
    knnExp = []
    for key in range(1,folds+1):
        knnExp += [ Experimento(
            f"./Data/{folds}_fold-SET-{key}-train.csv",
            f"./Data/{folds}_fold-SET-{key}-validate.csv",
            f"./resultados/{folds}_fold-SET-{key}-{k}-vecinos-resultados.csv",
            "0",
            f"{k}",
            "0","0","0"
        ) for k in range(1, maxK+1, step)]
    return knnExp
    

#prepareDataSet(10000, 5)
#experimentos += generarExperimentosKnn(5, 2001, 50)

prepareDataSet(42000, 5)
experimentos += generarExperimentosKnn(5, 51, 5)
#experimentos += generarExperimentosKnn(5, 6, 1)

<class 'pandas.core.frame.DataFrame'>


In [60]:
columnas=["SetEntrenamiento", "SetTesting", "Resultados", "Método", "Vecinos", "Numero de iteraciones", "Alfa", "Epsilon", "Tiempo"]
filas=[]
iteraciones = 1; #CAMBIAR CUANDO SE CORRA EN SERIO
for index, experimento in enumerate(experimentos):
    clear_output(wait=True)
    display(f"Experimento: {index+1} de {len(experimentos)} info:")
    experimento.display()
    
    tiempos = []
    for i in range(0, iteraciones):
        tiempo = correr_experimento(
            experimento.trainingDataset, 
            experimento.testingDataset, 
            experimento.outputFile, 
            experimento.vecinos, 
            experimento.metodo, 
            experimento.num_iter, 
            experimento.epsilon, 
            experimento.alfa
        )
        tiempos.append(tiempo)
        
        
    media = np.median(tiempos)
    if experimento.metodo == 0:
        filas.append([experimento.trainingDataset, experimento.testingDataset, experimento.outputFile, experimento.metodo, experimento.vecinos, tiempo])
    else:
        filas.append([experimento.trainingDataset, experimento.testingDataset, experimento.outputFile, experimento.metodo, experimento.vecinos, experimento.num_iter, experimento.alfa, experimento.epsilon, tiempo])
    
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("./resultados/resultado.csv", index=False, header=True);


'Experimento: 2 de 55 info:'

'Archivo De Entrenamiento: ./Data/5_fold-SET-1-train.csv'

'Archivo De Prueba: ./Data/5_fold-SET-1-validate.csv'

'Archivo De Salida: ./resultados/5_fold-SET-1-6-vecinos-resultados.csv'

'Metodo: 0'

'Vecinos: 6'

'Numero De Iteraciones: 0'

'Alfa: 0'

'Epsilon: 0'

KeyboardInterrupt: 

In [56]:
def extractLabelsFrom(archivo):
    dataset = pd.read_csv(archivo);
    labels = dataset[dataset.columns[0]]
    return labels


def calculateAccuracy(sourceFile, resultsFile):
    expectedLabels = extractLabelsFrom(sourceFile)
    resultLabels = extractLabelsFrom(resultsFile)
    
    hits = 0
    for index, result in enumerate(resultLabels):
        if result == expectedLabels[index]:
            hits += 1
    accuracy = hits/len(expectedLabels)
    return accuracy

def averageAccuracy(vecinos, x_fold):
    sourceFiles = [f"./Data/{x_fold}_fold-SET-{set}-validate.csv" for set in range(1, x_fold+1)]
    resultFiles = [f"./resultados/{x_fold}_fold-SET-{set}-{vecinos}-vecinos-resultados.csv" for set in range(1, x_fold+1)]
    
    accuracies = []
    for index, srcFile in enumerate(sourceFiles):
        accuracies.append(calculateAccuracy(srcFile, resultFiles[index]))
    
    return np.median(accuracies)

max_K = 1
max_accuracy = 0
for i in range (1,2002,50):
    next_value = averageAccuracy(i, 5)
    if next_value > max_accuracy:
        max_K = i
        max_accuracy = next_value

print(max_K)

1
